# ✍️ Module 7.2 - GPT et Génération de Texte avec TensorFlow

## 🎯 Objectifs
- Comprendre l'architecture GPT et la génération autoregressive
- Implémenter GPT from scratch avec TensorFlow
- Fine-tuner GPT pour génération de contenu spécialisé
- Explorer les techniques de génération avancées

## 📚 Contenu
1. **Architecture GPT** - Décodeur Transformer autorégressif
2. **Génération autoregressive** - Mécanismes et techniques
3. **Fine-tuning GPT** - Adaptation pour contenu spécialisé
4. **Techniques avancées** - Sampling, beam search, contrôle
5. **Applications pratiques** - Chatbot, génération créative

In [ ]:
# 📦 Installation des dépendances
!pip install tensorflow transformers datasets tokenizers matplotlib plotly openai

In [ ]:
# 📚 Imports
import tensorflow as tf
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
from transformers import TFGPT2LMHeadModel
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from datasets import load_dataset
import warnings
warnings.filterwarnings('ignore')

print(f"🔥 TensorFlow version: {tf.__version__}")
print(f"🤗 Transformers library loaded")
print(f"🎯 GPU Available: {tf.config.list_physical_devices('GPU')}")

## 🏗️ 1. Architecture GPT - Implémentation Détaillée

In [ ]:
class GPTCausalSelfAttention(tf.keras.layers.Layer):
    """👁️ Attention causale (masquée) pour GPT"""
    
    def __init__(self, config, **kwargs):
        super().__init__(**kwargs)
        self.num_heads = config.num_attention_heads
        self.head_dim = config.hidden_size // config.num_attention_heads
        self.embed_dim = config.hidden_size
        
        # 🎯 Projections Q, K, V combinées pour efficacité
        self.c_attn = tf.keras.layers.Dense(
            3 * self.embed_dim, name="c_attn"
        )
        self.c_proj = tf.keras.layers.Dense(
            self.embed_dim, name="c_proj"
        )
        
        self.attn_dropout = tf.keras.layers.Dropout(config.attn_pdrop)
        self.resid_dropout = tf.keras.layers.Dropout(config.resid_pdrop)
        
    def build(self, input_shape):
        # 🎭 Masque causal (triangulaire inférieur)
        max_len = 1024  # Longueur max pour le masque
        causal_mask = tf.linalg.band_part(
            tf.ones((max_len, max_len)), -1, 0
        )
        self.causal_mask = tf.Variable(
            causal_mask, trainable=False, name="causal_mask"
        )
        super().build(input_shape)
    
    def call(self, x, training=False):
        batch_size, seq_len, embed_dim = tf.shape(x)[0], tf.shape(x)[1], tf.shape(x)[2]
        
        # 🎯 Calcul Q, K, V
        qkv = self.c_attn(x)
        q, k, v = tf.split(qkv, 3, axis=-1)
        
        # 🔄 Reshape pour multi-head
        q = tf.reshape(q, (batch_size, seq_len, self.num_heads, self.head_dim))
        k = tf.reshape(k, (batch_size, seq_len, self.num_heads, self.head_dim))
        v = tf.reshape(v, (batch_size, seq_len, self.num_heads, self.head_dim))
        
        q = tf.transpose(q, [0, 2, 1, 3])  # (batch, heads, seq, head_dim)
        k = tf.transpose(k, [0, 2, 1, 3])
        v = tf.transpose(v, [0, 2, 1, 3])
        
        # 📊 Scores d'attention
        attn_scores = tf.matmul(q, k, transpose_b=True)
        attn_scores = attn_scores / tf.math.sqrt(tf.cast(self.head_dim, tf.float32))
        
        # 🎭 Application du masque causal
        mask = self.causal_mask[:seq_len, :seq_len]
        mask = tf.cast(mask, tf.float32)
        attn_scores = attn_scores * mask + (1.0 - mask) * (-1e9)
        
        # 🔥 Softmax
        attn_probs = tf.nn.softmax(attn_scores, axis=-1)
        attn_probs = self.attn_dropout(attn_probs, training=training)
        
        # 🎯 Application attention
        attn_output = tf.matmul(attn_probs, v)
        attn_output = tf.transpose(attn_output, [0, 2, 1, 3])
        attn_output = tf.reshape(attn_output, (batch_size, seq_len, embed_dim))
        
        # 🔄 Projection finale
        output = self.c_proj(attn_output)
        output = self.resid_dropout(output, training=training)
        
        return output, attn_probs

print("✅ GPTCausalSelfAttention implémenté")

In [ ]:
class GPTBlock(tf.keras.layers.Layer):
    """🏗️ Bloc Transformer GPT complet"""
    
    def __init__(self, config, **kwargs):
        super().__init__(**kwargs)
        self.attn = GPTCausalSelfAttention(config)
        
        # 🔄 Layer Normalization (pré-attention)
        self.ln_1 = tf.keras.layers.LayerNormalization(
            epsilon=config.layer_norm_epsilon, name="ln_1"
        )
        
        # 🧠 Feed Forward Network
        self.mlp_c_fc = tf.keras.layers.Dense(
            4 * config.hidden_size, activation="gelu", name="mlp_c_fc"
        )
        self.mlp_c_proj = tf.keras.layers.Dense(
            config.hidden_size, name="mlp_c_proj"
        )
        self.mlp_dropout = tf.keras.layers.Dropout(config.resid_pdrop)
        
        # 🔄 Layer Normalization (pré-MLP)
        self.ln_2 = tf.keras.layers.LayerNormalization(
            epsilon=config.layer_norm_epsilon, name="ln_2"
        )
    
    def call(self, x, training=False):
        # 👁️ Self-attention avec connexion résiduelle
        attn_output, attn_probs = self.attn(self.ln_1(x), training=training)
        x = x + attn_output
        
        # 🧠 MLP avec connexion résiduelle  
        mlp_output = self.mlp_c_fc(self.ln_2(x))
        mlp_output = self.mlp_c_proj(mlp_output)
        mlp_output = self.mlp_dropout(mlp_output, training=training)
        x = x + mlp_output
        
        return x, attn_probs

print("✅ GPTBlock implémenté")

In [ ]:
class SimpleGPT(tf.keras.Model):
    """🤖 Modèle GPT simplifié pour démonstration"""
    
    def __init__(self, config, **kwargs):
        super().__init__(**kwargs)
        self.config = config
        
        # 📝 Embeddings
        self.wte = tf.keras.layers.Embedding(
            config.vocab_size, config.hidden_size, name="wte"
        )
        self.wpe = tf.keras.layers.Embedding(
            config.max_position_embeddings, config.hidden_size, name="wpe"
        )
        self.drop = tf.keras.layers.Dropout(config.embd_pdrop)
        
        # 🏗️ Couches Transformer
        self.h = [GPTBlock(config, name=f"h_{i}") 
                  for i in range(config.num_hidden_layers)]
        
        # 🔄 Layer norm finale
        self.ln_f = tf.keras.layers.LayerNormalization(
            epsilon=config.layer_norm_epsilon, name="ln_f"
        )
        
        # 📊 Tête de génération (Language Modeling Head)
        self.lm_head = tf.keras.layers.Dense(
            config.vocab_size, use_bias=False, name="lm_head"
        )
    
    def call(self, input_ids, training=False):
        seq_len = tf.shape(input_ids)[1]
        
        # 📍 Position IDs
        position_ids = tf.range(seq_len, dtype=tf.int32)[tf.newaxis, :]
        
        # 📝 Embeddings
        token_embeds = self.wte(input_ids)
        position_embeds = self.wpe(position_ids)
        
        hidden_states = token_embeds + position_embeds
        hidden_states = self.drop(hidden_states, training=training)
        
        # 🏗️ Passage dans les couches Transformer
        attention_weights = []
        for layer in self.h:
            hidden_states, attn_probs = layer(hidden_states, training=training)
            attention_weights.append(attn_probs)
        
        # 🔄 Layer norm finale
        hidden_states = self.ln_f(hidden_states)
        
        # 📊 Génération des logits
        logits = self.lm_head(hidden_states)
        
        return logits, attention_weights

print("✅ SimpleGPT implémenté")

## 🎯 2. Chargement et Configuration de GPT-2

In [ ]:
# 🤖 Chargement de GPT-2 pré-entraîné
model_name = "gpt2"  # Ou "gpt2-medium", "gpt2-large"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = TFGPT2LMHeadModel.from_pretrained(model_name)

# 🔧 Configuration du tokenizer
tokenizer.pad_token = tokenizer.eos_token

print(f"🤖 Modèle GPT-2: {model_name}")
print(f"📏 Vocabulaire: {len(tokenizer)} tokens")
print(f"📊 Paramètres: {model.count_params():,}")
print(f"🎯 Max longueur: {tokenizer.model_max_length}")

# 📊 Configuration détaillée
config = model.config
print(f"\n🏗️ Architecture GPT-2:")
print(f"  - Couches: {config.n_layer}")
print(f"  - Têtes attention: {config.n_head}")
print(f"  - Dimension cachée: {config.n_embd}")
print(f"  - Contexte max: {config.n_positions}")

## ✍️ 3. Génération de Texte - Techniques et Stratégies

In [ ]:
def generate_text(prompt, model, tokenizer, max_length=100, temperature=0.7, 
                  top_k=50, top_p=0.95, repetition_penalty=1.1):
    """✍️ Génération de texte avec contrôles avancés"""
    
    # 🔄 Tokenisation du prompt
    input_ids = tokenizer.encode(prompt, return_tensors='tf')
    
    # ⚙️ Configuration de génération
    generation_config = {
        'max_length': max_length,
        'temperature': temperature,
        'top_k': top_k,
        'top_p': top_p,
        'repetition_penalty': repetition_penalty,
        'do_sample': True,
        'pad_token_id': tokenizer.eos_token_id,
        'eos_token_id': tokenizer.eos_token_id,
        'return_dict_in_generate': True,
        'output_scores': True
    }
    
    # 🚀 Génération
    with tf.device('/CPU:0'):  # CPU pour éviter les problèmes de mémoire
        outputs = model.generate(input_ids, **generation_config)
    
    # 📝 Décodage
    generated_text = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
    
    return {
        'prompt': prompt,
        'generated_text': generated_text,
        'new_text': generated_text[len(prompt):],
        'total_length': len(outputs.sequences[0]),
        'config': generation_config
    }

# 🧪 Test de génération
prompt = "L'intelligence artificielle va révolutionner"
result = generate_text(prompt, model, tokenizer, max_length=150)

print("🧪 Test de génération GPT-2:")
print("=" * 50)
print(f"📝 Prompt: {result['prompt']}")
print(f"✍️ Texte généré: {result['new_text']}")
print(f"📊 Longueur totale: {result['total_length']} tokens")

In [ ]:
# 🎛️ Comparaison des techniques de génération
def compare_generation_strategies(prompt, model, tokenizer):
    """⚖️ Comparaison de différentes stratégies de génération"""
    
    strategies = [
        {'name': 'Greedy', 'params': {'temperature': 0.1, 'top_k': 1, 'top_p': 1.0}},
        {'name': 'Low Temp', 'params': {'temperature': 0.3, 'top_k': 50, 'top_p': 0.95}},
        {'name': 'Balanced', 'params': {'temperature': 0.7, 'top_k': 50, 'top_p': 0.95}},
        {'name': 'Creative', 'params': {'temperature': 1.0, 'top_k': 100, 'top_p': 0.9}},
        {'name': 'Very Creative', 'params': {'temperature': 1.3, 'top_k': 0, 'top_p': 0.8}}
    ]
    
    results = []
    
    for strategy in strategies:
        result = generate_text(
            prompt, model, tokenizer, 
            max_length=100,
            **strategy['params']
        )
        results.append({
            'strategy': strategy['name'],
            'text': result['new_text'],
            'params': strategy['params']
        })
    
    return results

# 🧪 Test des stratégies
prompt = "Dans le futur, les robots"
comparison_results = compare_generation_strategies(prompt, model, tokenizer)

print("⚖️ Comparaison des stratégies de génération:")
print("=" * 60)
print(f"📝 Prompt: {prompt}")
print()

for i, result in enumerate(comparison_results, 1):
    print(f"🎯 Stratégie {i}: {result['strategy']}")
    print(f"   Paramètres: {result['params']}")
    print(f"   Texte: {result['text'][:150]}...")
    print()

## 🎨 4. Visualisation des Patterns d'Attention

In [ ]:
def visualize_attention_patterns(text, model, tokenizer, layer_idx=5, head_idx=0):
    """🎨 Visualisation des patterns d'attention GPT"""
    
    # 🔄 Tokenisation
    inputs = tokenizer(text, return_tensors='tf', max_length=50, truncation=True)
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    
    # 🎯 Forward pass avec attention
    outputs = model(inputs['input_ids'], output_attentions=True)
    attentions = outputs.attentions
    
    # 📊 Extraction de la matrice d'attention
    attention_matrix = attentions[layer_idx][0, head_idx].numpy()
    
    # 🎨 Visualisation interactive
    fig = px.imshow(
        attention_matrix,
        x=tokens,
        y=tokens,
        title=f"🎯 Attention Patterns - Couche {layer_idx}, Tête {head_idx}",
        labels=dict(x="Tokens (To)", y="Tokens (From)", color="Attention"),
        color_continuous_scale='Viridis'
    )
    
    fig.update_layout(
        width=800,
        height=800,
        xaxis={'side': 'top'},
        font_size=10
    )
    
    fig.show()
    
    return attention_matrix, tokens

# 🧪 Test de visualisation
test_text = "The artificial intelligence will transform our society in many ways"
attention_matrix, tokens = visualize_attention_patterns(test_text, model, tokenizer)

print(f"🎨 Visualisation pour: {test_text}")
print(f"📊 Matrice d'attention: {attention_matrix.shape}")
print(f"🔤 Tokens: {len(tokens)}")

## 🎯 5. Fine-tuning GPT pour Génération Spécialisée

In [ ]:
# 📊 Préparation d'un dataset pour fine-tuning
# Exemple: génération de descriptions de produits

def create_product_descriptions_dataset():
    """📦 Création d'un dataset synthétique pour descriptions produits"""
    
    products = [
        "Smartphone Samsung Galaxy S23 Ultra 256GB",
        "MacBook Pro 16 pouces M2 Max", 
        "Casque audio Sony WH-1000XM5",
        "Nike Air Max 270 React",
        "Parfum Chanel N°5 Eau de Toilette",
        "Montre Apple Watch Series 8",
        "Cuisine équipée IKEA moderne",
        "Tesla Model 3 Performance"
    ]
    
    descriptions = [
        "Un smartphone révolutionnaire avec appareil photo 200MP, écran Dynamic AMOLED et processeur Snapdragon ultra-puissant. Parfait pour la photographie professionnelle et les performances gaming.",
        "L'ordinateur portable le plus puissant d'Apple avec puce M2 Max, écran Liquid Retina XDR et autonomie exceptionnelle. Idéal pour les créatifs et professionnels.",
        "Casque à réduction de bruit leader du marché offrant une qualité audio exceptionnelle et un confort toute la journée. Technology LDAC pour audio haute résolution.",
        "Chaussures de sport alliant style et performance avec technologie React pour un amorti optimal. Design moderne et coloris tendance pour tous styles.",
        "Le parfum iconique français aux notes florales aldehydées. Une fragrance intemporelle qui incarne l'élégance et la sophistication féminine.",
        "Montre connectée la plus avancée avec capteurs de santé complets, GPS intégré et résistance à l'eau. Compatible avec tout l'écosystème Apple.",
        "Solution cuisine complète avec électroménager intégré, plan de travail quartz et rangements optimisés. Design scandinave et fonctionnalité maximale.",
        "Véhicule électrique haute performance avec autonomie 500km, accélération 0-100 en 3.3s et conduite autonome. L'avenir de la mobilité premium."
    ]
    
    # 🔄 Format pour fine-tuning
    training_data = []
    for product, description in zip(products, descriptions):
        # Format: "Produit: [PRODUIT] Description: [DESCRIPTION]"
        text = f"Produit: {product}\nDescription: {description}<|endoftext|>"
        training_data.append(text)
    
    return training_data

# 📦 Création du dataset
training_texts = create_product_descriptions_dataset()

print("📦 Dataset de descriptions produits créé:")
print(f"📊 Nombre d'exemples: {len(training_texts)}")
print("\n📝 Exemple:")
print(training_texts[0])

In [ ]:
# 🎯 Fine-tuning GPT-2 (version simplifiée pour démonstration)
def prepare_fine_tuning_data(texts, tokenizer, max_length=512):
    """🔄 Préparation des données pour fine-tuning"""
    
    input_ids = []
    attention_masks = []
    
    for text in texts:
        encoded = tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=max_length,
            return_tensors='tf'
        )
        
        input_ids.append(encoded['input_ids'][0])
        attention_masks.append(encoded['attention_mask'][0])
    
    return {
        'input_ids': tf.stack(input_ids),
        'attention_mask': tf.stack(attention_masks)
    }

# 🔄 Préparation données
fine_tune_data = prepare_fine_tuning_data(training_texts, tokenizer, max_length=256)

print("🔄 Données de fine-tuning préparées:")
print(f"📊 Shape input_ids: {fine_tune_data['input_ids'].shape}")
print(f"📊 Shape attention_mask: {fine_tune_data['attention_mask'].shape}")

# ⚙️ Configuration pour fine-tuning
optimizer = tf.keras.optimizers.Adam(
    learning_rate=5e-5,  # Learning rate plus élevé que BERT
    epsilon=1e-08
)

# 📊 Loss pour language modeling
def compute_loss(labels, logits):
    """📊 Calcul de la loss pour language modeling"""
    # Shift labels for causal LM
    shift_labels = labels[..., 1:]
    shift_logits = logits[..., :-1, :]
    
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none'
    )
    
    loss = loss_fn(shift_labels, shift_logits)
    
    # Masquer les tokens de padding
    mask = tf.cast(shift_labels != tokenizer.pad_token_id, tf.float32)
    loss = loss * mask
    
    return tf.reduce_sum(loss) / tf.reduce_sum(mask)

print("✅ Configuration fine-tuning prête")

## 🚀 6. Applications Pratiques - Génération Contrôlée

In [ ]:
def generate_product_description(product_name, model, tokenizer):
    """📦 Générateur de descriptions produits"""
    
    prompt = f"Produit: {product_name}\nDescription:"
    
    result = generate_text(
        prompt, model, tokenizer,
        max_length=200,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1
    )
    
    # 🔍 Extraction de la description
    full_text = result['generated_text']
    if "Description:" in full_text:
        description = full_text.split("Description:")[1].strip()
        # Nettoyer et limiter
        description = description.split('\n')[0]  # Première ligne
        description = description.replace('<|endoftext|>', '').strip()
    else:
        description = "Description non générée"
    
    return {
        'product': product_name,
        'description': description,
        'prompt': prompt,
        'full_generation': full_text
    }

# 🧪 Tests de génération de descriptions
test_products = [
    "iPhone 15 Pro Max 512GB",
    "Gaming Chair DXRacer Formula",
    "Machine à café Nespresso Vertuo",
    "Drone DJI Mini 3 Pro",
    "Guitare électrique Fender Stratocaster"
]

print("📦 Génération de descriptions produits:")
print("=" * 60)

for i, product in enumerate(test_products, 1):
    result = generate_product_description(product, model, tokenizer)
    print(f"\n🛍️ Produit {i}: {result['product']}")
    print(f"📝 Description: {result['description']}")
    print("-" * 40)

In [ ]:
def creative_story_generator(theme, style, model, tokenizer):
    """📚 Générateur d'histoires créatives"""
    
    style_prompts = {
        "science-fiction": "Dans un futur lointain où la technologie a transformé l'humanité,",
        "fantasy": "Dans un royaume magique peuplé de créatures fantastiques,",
        "mystery": "Par une nuit sombre et pluvieuse, l'inspecteur découvrit un mystère troublant:",
        "romance": "C'était un matin de printemps quand leurs regards se croisèrent pour la première fois,",
        "adventure": "L'aventure commença quand ils trouvèrent la carte ancienne dans le grenier,"
    }
    
    base_prompt = style_prompts.get(style, "Il était une fois,")
    prompt = f"{base_prompt} {theme}"
    
    # 🎨 Paramètres créatifs
    result = generate_text(
        prompt, model, tokenizer,
        max_length=300,
        temperature=0.8,  # Plus créatif
        top_p=0.9,
        top_k=80,
        repetition_penalty=1.2
    )
    
    return {
        'theme': theme,
        'style': style,
        'story': result['generated_text'],
        'word_count': len(result['generated_text'].split())
    }

# 🎭 Tests de génération créative
creative_tests = [
    {"theme": "un robot qui découvre l'amour", "style": "science-fiction"},
    {"theme": "un dragon qui a peur du feu", "style": "fantasy"},
    {"theme": "une disparition mystérieuse dans un petit village", "style": "mystery"}
]

print("🎭 Génération d'histoires créatives:")
print("=" * 60)

for i, test in enumerate(creative_tests, 1):
    story = creative_story_generator(test['theme'], test['style'], model, tokenizer)
    print(f"\n📚 Histoire {i} ({story['style']})")
    print(f"🎯 Thème: {story['theme']}")
    print(f"📝 Début: {story['story'][:200]}...")
    print(f"📊 Mots: {story['word_count']}")
    print("-" * 40)

## 🎛️ 7. Contrôle Avancé de la Génération

In [ ]:
def analyze_generation_quality(texts, original_prompt):
    """📊 Analyse de la qualité des générations"""
    
    metrics = []
    
    for i, text in enumerate(texts):
        # 📏 Métriques basiques
        word_count = len(text.split())
        char_count = len(text)
        avg_word_length = np.mean([len(word) for word in text.split()])
        
        # 🔄 Répétition de mots
        words = text.lower().split()
        unique_words = len(set(words))
        repetition_ratio = unique_words / len(words) if words else 0
        
        # 📝 Cohérence avec le prompt
        prompt_words = set(original_prompt.lower().split())
        text_words = set(words)
        coherence = len(prompt_words & text_words) / len(prompt_words) if prompt_words else 0
        
        metrics.append({
            'index': i,
            'word_count': word_count,
            'char_count': char_count,
            'avg_word_length': avg_word_length,
            'repetition_ratio': repetition_ratio,
            'coherence': coherence,
            'text': text[:100] + "..." if len(text) > 100 else text
        })
    
    return metrics

# 🧪 Test avec différentes températures
prompt = "L'avenir de l'intelligence artificielle"
temperatures = [0.3, 0.7, 1.0, 1.3]

generation_results = []
for temp in temperatures:
    result = generate_text(
        prompt, model, tokenizer,
        max_length=150,
        temperature=temp,
        top_p=0.9
    )
    generation_results.append(result['new_text'])

# 📊 Analyse des résultats
quality_metrics = analyze_generation_quality(generation_results, prompt)

print("📊 Analyse de qualité selon la température:")
print("=" * 60)

for i, (temp, metrics) in enumerate(zip(temperatures, quality_metrics)):
    print(f"\n🌡️ Température: {temp}")
    print(f"  📝 Mots: {metrics['word_count']}")
    print(f"  🔄 Ratio diversité: {metrics['repetition_ratio']:.2f}")
    print(f"  🎯 Cohérence: {metrics['coherence']:.2f}")
    print(f"  📖 Texte: {metrics['text']}")

In [ ]:
# 🎨 Visualisation des métriques de qualité
import pandas as pd

# 📊 Création du DataFrame
df_metrics = pd.DataFrame([
    {
        'Temperature': temp,
        'Word Count': metrics['word_count'],
        'Diversity Ratio': metrics['repetition_ratio'],
        'Coherence': metrics['coherence'],
        'Avg Word Length': metrics['avg_word_length']
    }
    for temp, metrics in zip(temperatures, quality_metrics)
])

# 📈 Graphiques interactifs
fig = go.Figure()

# Diversité
fig.add_trace(go.Scatter(
    x=df_metrics['Temperature'],
    y=df_metrics['Diversity Ratio'],
    mode='lines+markers',
    name='Diversité',
    line=dict(color='blue', width=3),
    marker=dict(size=8)
))

# Cohérence
fig.add_trace(go.Scatter(
    x=df_metrics['Temperature'],
    y=df_metrics['Coherence'],
    mode='lines+markers',
    name='Cohérence',
    line=dict(color='red', width=3),
    marker=dict(size=8)
))

fig.update_layout(
    title="📊 Impact de la Température sur la Qualité de Génération",
    xaxis_title="Température",
    yaxis_title="Score (0-1)",
    hovermode='x unified',
    width=800,
    height=500
)

fig.show()

print("📊 Observations:")
print(f"  🎯 Meilleure diversité à T={temperatures[df_metrics['Diversity Ratio'].idxmax()]}")
print(f"  🎯 Meilleure cohérence à T={temperatures[df_metrics['Coherence'].idxmax()]}")
print(f"  💡 Compromis optimal autour de T=0.7-1.0")

## 🎉 Conclusion et Applications Pratiques

### ✅ Ce que nous avons accompli:
1. **Architecture GPT** - Implémentation détaillée avec attention causale
2. **Génération autoregressive** - Techniques et paramètres de contrôle
3. **Fine-tuning** - Adaptation pour génération spécialisée
4. **Visualisation** - Patterns d'attention et métriques de qualité
5. **Applications** - Génération créative et contrôlée

### 🎯 Techniques de génération maîtrisées:
- **Temperature control** - Balance créativité/cohérence
- **Top-k/Top-p sampling** - Contrôle de la diversité
- **Repetition penalty** - Éviter les répétitions
- **Prompt engineering** - Guide la génération

### 🚀 Applications pratiques:
1. **Génération de contenu** - Marketing, descriptions produits
2. **Assistant créatif** - Écriture, brainstorming
3. **Chatbot intelligent** - Conversations naturelles
4. **Complétion de code** - Assistance programmation

### 💡 Bonnes pratiques:
- **Température 0.7-1.0** pour équilibre créativité/cohérence
- **Top-p 0.9** pour diversité contrôlée
- **Repetition penalty 1.1-1.2** pour éviter les boucles
- **Prompt engineering** crucial pour résultats qualité
- **Fine-tuning** sur données spécialisées améliore drastiquement

### 🔮 Évolutions futures:
- **Modèles plus gros** (GPT-4, Claude, LLaMA)
- **Multimodalité** (texte + images)
- **Agents autonomes** avec tools/APIs
- **Optimisations** (quantification, distillation)
- **Contrôle éthique** renforcé